<a href="https://colab.research.google.com/github/hyounghe0724/StartPytorch/blob/main/Cross_Vaildation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/My Drive/GoogleColab/deeplearningbrov2/pytorch

/content/gdrive/My Drive/GoogleColab/deeplearningbrov2/pytorch


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # 전체 데이터를 학습 데이터와 평가 데이터로 나눈다.\
import torch
from torch import nn, optim # torch 내의 세부적인 기능을 불러온다. (신경망 기술, 손실함수, 최적화 방법 등)
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리
import torch.nn.functional as F # torch 내의 세부적인 기능을 불러온다. (신경망 기술 등)

# Cross Validation
from sklearn.model_selection import KFold

# Plot
import matplotlib.pyplot as plt # 시각화 도구

In [ ]:
df = pd.read_csv('./data/reg.csv', index_col=[0])
# 데이터를 넘파이 배열로 만들기
X = df.drop("Price", axis=1).to_numpy() # target 값 price를 제외하고 넘파이 배열로 만듬
Y = df["Price"].to_numpy().reshape((-1,1)) # (n,) 1차원을 (n,1) 2차원으로 만들어준다.
# -1은 행의 갯수를 자동으로 계산, 1은 [[a], [b]..]처럼 col값이 1로 설정인다.


In [ ]:
# 텐서 데이터로 변환하는 클래스
class TensorData(Dataset):

  def __init__(self, x_data, y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.FloatTensor(y_data)
    self.len = self.y_data.shape[0]

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self,):
    return self.len



In [ ]:
X_train, X_test, Y_train, Y_test  = train_test_split(X,Y, test_size=0.7)
trainset = TensorData(X_train, Y_train)

testset = TensorData(X_test, Y_test)
testloader = DataLoader(testset, batch_size=32, shuffle=False)


In [ ]:
class Regressor(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(13,50,bias = True)
    self.fc2 = nn.Linear(50,30,bias = True)
    self.fc3 = nn.Linear(30,1,bias = True)
    self.dropout = nn.Dropout(0.2) # 연산이 될 때 마다 2e-1 비율로 랜덤하게 노드를 없앤다.
    # overfitting 방지
    # 네트워크의 일부만을 사용, 서로 다른 하위모델들이 동시에 학습되는 효과

  def forward(self, x):
    x= F.relu(self.fc1(x))
    x = self.dropout(F.relu(self.fc2(x))) # Hidden layer에만 사용 출력층에서 사용해서는 안된다
    x= F.relu(self.fc3(x))

    return x

In [ ]:
kfold = KFold(n_splits=3, shuffle=True)


In [ ]:
criterion = nn.MSELoss()

In [ ]:
def rmse(dataloader):
  with torch.no_grad():
    square_sum = 0
    num_instances = 0
    model.eval()
    for data in dataloader:
      inputs, targets = data
      outputs = model(inputs)
      square_sum += torch.sum((outputs - targets)**2).item()
      num_instances += len(targets)
  model.train()
  return np.sqrt(square_sum/num_instances)

In [ ]:
# 평가 모델들에 대해서 CV 진행
vaildation_loss = []
for fold, (train_idx, val_idx) in enumerate(kfold.split(trainset)):
  train_subsampler = SubsetRandomSampler(train_idx)
  val_subsmapler = SubsetRandomSampler(val_idx)

  trainloader = DataLoader(trainset, batch_size=32, sampler=train_subsampler)
  valloader = DataLoader(trainset, batch_size=32, sampler=val_subsmapler)

  model = Regressor()
  optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=3e-3)

  for epoch in range(200):

    for data in trainloader:

      inputs, values = data # X, Y
      optimizer.zero_grad() # 최적화 초기화

      outputs = model(inputs)
      loss = criterion(outputs, values)
      loss.backward()
      optimizer.step()

  train_rmse = rmse(trainloader)
  val_rmse = rmse(valloader)
  print("f-fold", fold, " Train Loss : %.4f, vaildation Loss: %.4f" %(train_rmse, val_rmse))
  vaildation_loss.append(val_rmse)

vaildation_loss = np.array(vaildation_loss)
mean = np.mean(vaildation_loss)
std = np.std(vaildation_loss)
print("Validation Score: %.4f, ± %.4f" %(mean, std))

f-fold 0  Train Loss : 0.1130, vaildation Loss: 0.1115
f-fold 1  Train Loss : 0.0928, vaildation Loss: 0.1534
f-fold 2  Train Loss : 0.1196, vaildation Loss: 0.1232
Validation Score: 0.1294, ± 0.0177


In [ ]:
# CV를 통해 best model를 정했다면 평가를 진행한다.
# 전체 학습 데이터를 이용한 DataLoader 정의
trainloader = DataLoader(trainset, batch_size=32)

# 모델
model = Regressor()
optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=3e-3)

for epoch in range(200): # 200번 학습을 진행한다.

    for data in trainloader: # 무작위로 섞인 32개 데이터가 있는 배치가 하나 씩 들어온다.

        inputs, values = data # data에는 X, Y가 들어있다.

        optimizer.zero_grad() # 최적화 초기화

        outputs = model(inputs) # 모델에 입력값 대입 후 예측값 산출
        loss = criterion(outputs, values) # 손실 함수 계산
        loss.backward() # 손실 함수 기준으로 역전파 설정
        optimizer.step() # 역전파를 진행하고 가중치 업데이트

In [ ]:
train_rmse = rmse(trainloader) # 학습 데이터의 RMSE
test_rmse = rmse(testloader) # 시험 데이터의 RMSE
print(" Train Loss: %.4f, Test Loss: %.4f" %(train_rmse, test_rmse))

 Train Loss: 0.1034, Test Loss: 0.1127
